# ***Library***

In [ ]:
#@title Default title text

import os
import numpy as np
!pip install wget
!pip install download
import urllib.parse
from download import download
from tqdm import tqdm
import os
import shutil
from IPython.display import clear_output
from tqdm import tqdm
!pip install gtts
from gtts import gTTS 

from natsort import natsorted # pip install natsort
import requests
import wget
from natsort import natsorted
%cd /content/drive/MyDrive/text_to_zip
clear_output()

# ***Split file***

In [ ]:
#@title
folder_path = r'big_text'    # thư mục chứa files text
save_path = r'mp3'      #thư mục lưu file sau khi cắt, sau này là mp3
size = 60                           # size*1024 tự trong mỗi file text( không tương đồng với kích thước)

"""LƯU LIST STRING RA TEXT : list string, thư mục save, tên file lưu 
Tại sao gộp đoạn code này vào hàm split thì gây lỗi, phải viết hàm riêng
"""
def save(a, save_path, str_name):
  os.mkdir(save_path + '/' + str_name)
  os.mkdir(save_path + '/' + str_name + '/' + str_name)
  for i in range(0, len(a)):
      f = open(save_path + '/' + str_name + '_' + str(i) + '.txt', 'w', encoding = 'utf-8')
      f.write(a[i]) 
      f = open(save_path + '/' + str_name + '/' + str_name + '/' + str(i).zfill(4) + '.txt', 'w', encoding = 'utf-8')
      f.write(a[i]) 
      f.close()


"""CUT NHỎ FILE TXT: link file, link folder save, kích thước tính theo kb
"""
def split(file_path, save_path, size, str_name): 
  f = open(file_path,encoding = 'utf-8')
  txt = f.read().replace('\n', ' ')
  f.close()
  str = txt.split('.')

  luu = []
  a = ''
  for i in range(len(str)):
      if len(a) < size * 1024:
          a = a + '. ' + str[i]
      else:
          a = 'Bắt đầu Bắt đầu Bắt đầu ' + a + ' Kết thúc Kết thúc Kết thúc '
          luu.append(a)
          a = str[i]
  a = 'Bắt đầu Bắt đầu Bắt đầu ' + a + 'Kết thúc Kết thúc Kết thúc '
  luu.append(a)
  save(luu, save_path, str_name)


""" CUT NHỎ NHIỀU FILE TRONG FOLDER:
"""
def split_files(folder_path, save_path, size):
  files = os.listdir(folder_path)
  for file in tqdm(files):
      file_path = folder_path + '/' + file
      split(file_path, save_path, size, file[:-4])
      os.remove(file_path)

split_files(folder_path, save_path, size)

# ***Convert text to mp3***

In [ ]:
#@title
def form_str(text):
  size = 30
  x = text.split(' ')
  them = (int(len(x)/size) + 1)*size - len(x)
  for i in range(them):
    x.append('')
  x = np.array(x)
  x = x.reshape(-1, 30)
  arr_text = []
  for i in x:
    text = ''
    for j in i:
      text = text + j + ' '
    arr_text.append(text.strip())
  return arr_text

In [ ]:
#@title
def xu_li_file(path):
  ###### chuyen ve mang nho hon 30
  dau = '''…,.?<>;:'"{}[]!()”“'''
  dau = list(dau)
  dau.append('\n')
  with open(path, 'r', encoding='utf-8') as f:
    text = f.read()
  for i in dau:
    text = text.replace(i, '.')
  a = text.split('.')
  x = []
  for i in a:
    if len(i.split(' ')) > 30:
      x = x + form_str(i)
    else:
      x.append(i)
  ######### chuyen ve mang lon nhat nho hon 30
  k = []
  text = ''
  for i in x:
    if len(i.split(' ')) + len(text.split(' ')) > 31:
      k.append(text)
      text = i
    else:
      text = text + '.' + i
  if text != '':
    k.append(text)
  return k

In [ ]:
#@title
def download_arr(arr):
  try:
    shutil.rmtree('/content/1')
  except:
    dadhaslkdjasdasl = 0
  try: 
    os.mkdir('/content/1')
  except:
    sadkasjdas = 0
  href = 'https://translate.googleapis.com/translate_tts?client=gtx&ie=UTF-8&tl=vi&q='
  for i in (range(len(arr))):
    path = os.path.join('/content/1', str(i).zfill(4))
    url = href + urllib.parse.quote(arr[i])
    try: 
      wget.download(url, path)
    except:
      try:
        url = 'https://translate.googleapis.com/translate_tts?client=gtx&ie=UTF-8&tl=vi&q=' + urllib.parse.quote(arr[i])
        wget.download(url, path)
      except:
        output = gTTS(text=arr[i], lang='vi', slow=False)
        output.save(path)

In [ ]:
#@title
def gop_file(save):
  f = os.listdir('/content/1')
  f.sort()
  for i in f:
    path_file = os.path.join('/content/1', i)
    with open(path_file, 'rb') as q:
      a = q.read() 
    with open(save, 'a+b') as q:
      q.write(a)

In [ ]:
#@title
def create_mp3(path, path_save):
  try:
    os.remove(path_save)
  except:
    akahkak = 0
  a = xu_li_file(path)
  download_arr(a)
  gop_file(path_save)

In [ ]:
#@title
path = 'mp3'
a = []
for i in os.listdir(path):
  n = os.path.join(path, i)
  if os.path.isfile(n) and i.endswith('.txt'):
    a.append(i)

a = natsorted(a)
for i in tqdm(a):
  path_i = os.path.join(path, i)
  save_i = os.path.join(path, i[:-4]+'.mp3')
  create_mp3(path_i, save_i)
  os.remove(path_i)
  clear_output()

# **Nén**

In [ ]:
#@title
folder_path = r'mp3'
pdf_path = 'pdf'
save_zip = 'zip'

# lấy folder
folders = []
for i in os.listdir(folder_path):
  if not os.path.isfile(os.path.join(folder_path, i)):
    folders.append(i)

# di chuyển pdf vào thư mục đúng
for i in folders:
  try:
    o = os.path.join(pdf_path, i + '.pdf')
    n = os.path.join(folder_path, i, i + '.pdf')
    os.rename(o, n)
  except:
    sa = 0

# di chuyển mp3 vào thư mục đúng
for i in folders:
  for j in os.listdir(folder_path):
    o = os.path.join(folder_path, j)
    if j.find(i) > -1 and j!=i:
      n = os.path.join(folder_path, i, j[:-4].split('_')[-1].zfill(4) + '.mp3')
      os.rename(o, n)

# nen file 
for folder in tqdm(folders):
  shutil.make_archive(save_zip + '/' + folder, 'zip', folder_path + '/' + folder)
  shutil.rmtree(folder_path + '/' + folder)

In [ ]:
#@title
shutil.rmtree(folder_path)
os.mkdir(folder_path)
shutil.rmtree(pdf_path)
os.mkdir(pdf_path)

# ***Javascript***

In [ ]:
#@title
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
setInterval(ClickConnect, 60000)